## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_counts = application_df['APPLICATION_TYPE'].value_counts()

application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classification_counts = application_df['CLASSIFICATION'].value_counts()

classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
data_dummies = pd.get_dummies(application_df)
data_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [6]:
# Split our preprocessed data into our features and target arrays
X = data_dummies.drop(['IS_SUCCESSFUL'], axis=1)
y = data_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

## Optimization Attempt #1
### Third hidden layer and increasing nodes

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 60)                4860      
                                                                 
 dense_3 (Dense)             (None, 1)                 61        
                                                                 
Total params: 17401 (67.97 KB)
Trainable params: 17401 (67.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 498us/step - loss: 0.5707 - accuracy: 0.7232
Epoch 2/100
804/804 [==============================] - 0s 475us/step - loss: 0.5551 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 0s 478us/step - loss: 0.5515 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 0s 479us/step - loss: 0.5501 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 0s 489us/step - loss: 0.5492 - accuracy: 0.7340
Epoch 6/100
804/804 [==============================] - 0s 465us/step - loss: 0.5476 - accuracy: 0.7344
Epoch 7/100
804/804 [==============================] - 0s 479us/step - loss: 0.5472 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 0s 499us/step - loss: 0.5469 - accuracy: 0.7358
Epoch 9/100
804/804 [==============================] - 0s 479us/step - loss: 0.5457 - accuracy: 0.7364
Epoch 10/100
804/804 [==============================] - 0s 497us/step - l

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5664 - accuracy: 0.7247 - 158ms/epoch - 590us/step
Loss: 0.5663895606994629, Accuracy: 0.7246647477149963


## Optimization Attempt #2
### Third hidden layer, increasing nodes, and tanh activation

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features_2 = len(X_train_scaled[0])
hidden_nodes_layer1_2 =  100
hidden_nodes_layer2_2 = 80
hidden_nodes_layer3_2 = 60

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features_2, activation="tanh")
)

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="tanh"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="tanh"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4400      
                                                                 
 dense_5 (Dense)             (None, 80)                8080      
                                                                 
 dense_6 (Dense)             (None, 60)                4860      
                                                                 
 dense_7 (Dense)             (None, 1)                 61        
                                                                 
Total params: 17401 (67.97 KB)
Trainable params: 17401 (67.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_2 = nn_2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 479us/step - loss: 0.5721 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 0s 477us/step - loss: 0.5591 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 0s 478us/step - loss: 0.5558 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 0s 502us/step - loss: 0.5526 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 0s 479us/step - loss: 0.5506 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 0s 481us/step - loss: 0.5499 - accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 0s 481us/step - loss: 0.5484 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 0s 486us/step - loss: 0.5477 - accuracy: 0.7333
Epoch 9/100
804/804 [==============================] - 0s 467us/step - loss: 0.5461 - accuracy: 0.7354
Epoch 10/100
804/804 [==============================] - 0s 480us/step - l

In [13]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7243 - 140ms/epoch - 521us/step
Loss: 0.558255672454834, Accuracy: 0.7243148684501648


## Optimization Attempt #3
### Third hidden layer, decreasing nodes, and relu activation

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features_3 = len(X_train_scaled[0])
hidden_nodes_layer1_3 =  20
hidden_nodes_layer2_3 = 18
hidden_nodes_layer3_3 = 16

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_3, input_dim=number_input_features_3, activation="relu")
)

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_3, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 20)                880       
                                                                 
 dense_9 (Dense)             (None, 18)                378       
                                                                 
 dense_10 (Dense)            (None, 16)                304       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1579 (6.17 KB)
Trainable params: 1579 (6.17 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_3 = nn_3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 465us/step - loss: 0.5802 - accuracy: 0.7158
Epoch 2/100
804/804 [==============================] - 0s 446us/step - loss: 0.5571 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 0s 447us/step - loss: 0.5538 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 0s 455us/step - loss: 0.5519 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 0s 447us/step - loss: 0.5507 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 0s 454us/step - loss: 0.5498 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 0s 450us/step - loss: 0.5490 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 0s 452us/step - loss: 0.5485 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 0s 453us/step - loss: 0.5475 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 0s 458us/step - l

In [16]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7266 - 134ms/epoch - 501us/step
Loss: 0.5547957420349121, Accuracy: 0.7266472578048706


## Optimization Attempt #4
### Dropping Income Amount Column from Dataframe

In [17]:
application_df_4 = application_df.drop(columns=['INCOME_AMT'])

data_dummies_4 = pd.get_dummies(application_df_4)
data_dummies_4.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,1,0


In [18]:
# Split our preprocessed data into our features and target arrays
X_4 = data_dummies_4.drop(['IS_SUCCESSFUL'], axis=1)
y_4 = data_dummies_4['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4, random_state=78)

In [19]:
# Create a StandardScaler instances
scaler_4 = StandardScaler()

# Fit the StandardScaler
X_scaler_4 = scaler_4.fit(X_train_4)

# Scale the data
X_train_scaled_4 = X_scaler_4.transform(X_train_4)
X_test_scaled_4 = X_scaler_4.transform(X_test_4)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features_4 = len(X_train_scaled_4[0])
hidden_nodes_layer1_4 =  100
hidden_nodes_layer2_4 = 80
hidden_nodes_layer3_4 = 60

nn_4 = tf.keras.models.Sequential()

# First hidden layer
nn_4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_4, input_dim=number_input_features_4, activation="relu")
)

# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_4, activation="relu"))

# Third hidden layer
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_4, activation="relu"))

# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               3500      
                                                                 
 dense_13 (Dense)            (None, 80)                8080      
                                                                 
 dense_14 (Dense)            (None, 60)                4860      
                                                                 
 dense_15 (Dense)            (None, 1)                 61        
                                                                 
Total params: 16501 (64.46 KB)
Trainable params: 16501 (64.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the model
nn_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_4 = nn_4.fit(X_train_scaled_4,y_train_4,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 487us/step - loss: 0.5756 - accuracy: 0.7186
Epoch 2/100
804/804 [==============================] - 0s 479us/step - loss: 0.5638 - accuracy: 0.7241
Epoch 3/100
804/804 [==============================] - 0s 479us/step - loss: 0.5619 - accuracy: 0.7259
Epoch 4/100
804/804 [==============================] - 0s 476us/step - loss: 0.5601 - accuracy: 0.7274
Epoch 5/100
804/804 [==============================] - 0s 479us/step - loss: 0.5601 - accuracy: 0.7277
Epoch 6/100
804/804 [==============================] - 0s 499us/step - loss: 0.5591 - accuracy: 0.7276
Epoch 7/100
804/804 [==============================] - 0s 478us/step - loss: 0.5585 - accuracy: 0.7280
Epoch 8/100
804/804 [==============================] - 0s 487us/step - loss: 0.5576 - accuracy: 0.7286
Epoch 9/100
804/804 [==============================] - 0s 479us/step - loss: 0.5576 - accuracy: 0.7289
Epoch 10/100
804/804 [==============================] - 0s 481us/step - l

In [22]:
# Evaluate the model using the test data
model_loss_4, model_accuracy_4 = nn_4.evaluate(X_test_scaled_4,y_test_4,verbose=2)
print(f"Loss: {model_loss_4}, Accuracy: {model_accuracy_4}")

268/268 - 0s - loss: 0.5566 - accuracy: 0.7282 - 138ms/epoch - 514us/step
Loss: 0.5566387176513672, Accuracy: 0.7281632423400879


In [25]:
nn_4.save('HDF5_Files/AlphabetSoupCharity_Optimization.h5')